<a href="https://colab.research.google.com/github/lior9005/deep-learning/blob/main/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ADL final project


---


**Authors:**
> Lior Sharony 316380138


> Noam Zigler 314868019


> Eden Miran 208742429


In [8]:
# Imports
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
from torch import nn, optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset, random_split
from tqdm.auto import tqdm

# Eval mode
eval_mode = True

# Data & Preprocessing

In [12]:
# Datasets, dataloaders, transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    transforms.Resize((28, 28))
])

train_set = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_set = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
train_size = int(0.8 * len(train_set))  # 80% training
val_size = len(train_set) - train_size  # 20% validation
train_dataset, val_dataset = random_split(train_set, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

# Subset of CIFAR10/FashionMNIST
fashion_mnist = torchvision.datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)
cifar10 = torchvision.datasets.CIFAR10(root='./data', train=False, transform=transforms, download=True)

# Create dataloaders for OOD datasets
ood_loader_fashion = DataLoader(fashion_mnist, batch_size=64, shuffle=False)
ood_loader_cifar = DataLoader(cifar10, batch_size=64, shuffle=False)


Files already downloaded and verified


# Models

In [ ]:
# Baseline model class


# OSR model class



# Training

In [ ]:
if not (eval_mode)

  # Training procedure


  # Training and validation loss and accuracy for MNIST plot



# Evaluation - CIFAR10/Fashion-MNIST

In [ ]:
if (eval_mode)
  # Load the trained models weights


# Baseline results

**OSR approach:**

---



In [ ]:
# OOD results


# OSR results


# t-SNE and/or PCA visualization (optional)


# Additional figures (optional)

# Evaluation - Unknown Dataset

In [ ]:
# placeholder for the combined OSR dataset and dataloader


# Load the trained models weights


# OOD results


# OSR results